#**Preparacion del entorno**

## **Instalacion de librerias**

In [ ]:
# Bibliotecas básicas que se usan en el experimento
import pandas as pd
import random
import numpy as np
import time
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

#imblearn, de clase desequilibrada,
!pip install -U imbalanced-learn
from imblearn.over_sampling import RandomOverSampler, SMOTE, BorderlineSMOTE,SVMSMOTE, ADASYN #in the case of this experiment, no use kmeanssmote

#Liberia sklearn 
from sklearn.impute import SimpleImputer # Univariate imputer for completing missing values with simple strategies.
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, GridSearchCV, RandomizedSearchCV #defines the data partitioning and cross validation strategy. In this experiment use GridsearhCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler #In this expermiento, is used StandardScaler
from sklearn.pipeline import Pipeline

#Metodos de seleccion de caracteristicas
from sklearn.feature_selection import SelectKBest, f_classif, chi2, mutual_info_classif, GenericUnivariateSelect, VarianceThreshold, SequentialFeatureSelector, RFE, SelectFromModel, RFECV #In this expermiento, is used SelectKBest with f_classif and mutual_info_classif
!pip install skfeature-chappers
from skfeature.function.similarity_based import fisher_score, SPEC, lap_score, reliefF, trace_ratio # https://jundongl.github.io/scikit-feature/html/skfeature.function.html
from skfeature.function.information_theoretical_based import CIFE, CMIM, MRMR, MIM, JMI
from skfeature.function.statistical_based import t_score, gini_index, chi_square, f_score, low_variance
from scipy.stats import wilcoxon #This function is used in the feature selection method, according to the Wilcoxon criteria.
!pip install Boruta
from boruta import BorutaPy

# clasificadores
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC, NuSVC, SVC
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.dummy import DummyClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF,DotProduct,Matern,RationalQuadratic,WhiteKernel
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

#metricas
from sklearn.metrics import f1_score,precision_score,recall_score,accuracy_score,confusion_matrix,roc_auc_score, classification_report

#Guardar el dataframe y la serie en google DRIVE
from google.colab import drive, files, auth
from google.auth import default
drive.mount('/content/drive')


##### **Funcion de Wilcoxon**

In [ ]:
#método de selección de características tomado de github por Aydin demircioglu
#Link : https://github.com/aydindemircioglu/radFS/blob/main/extraFeatureSelections.py
def wilcoxon_score(X, y):
    """
    This function calculates t_score for each feature, where t_score is only used for binary problem
    t_score = |mean1-mean2|/sqrt(((std1^2)/n1)+((std2^2)/n2)))
    Input
    -----
    X: {numpy array}, shape (n_samples, n_features)
        input data
    y: {numpy array}, shape (n_samples,)
        input class labels
    Output
    ------
    F: {numpy array}, shape (n_features,)
        t-score for each feature
    """

    n_samples, n_features = X.shape
    F = np.zeros(n_features)
    c = np.unique(y)
    if len(c) == 2:
        for i in range(n_features):
            st, p = wilcoxon(X[:, i], y, zero_method='wilcox', correction=False, alternative='two-sided', mode='auto')
            F[i] = 1 - p
    else:
        print('y should be guaranteed to a binary class vector')
        exit(0)
    return np.abs(F)
def feature_ranking(F):
    """
    Rank features in descending order according to t-score, the higher the t-score, the more important the feature is
    """
    idx = np.argsort(F)
    return idx[::-1]


##**Funciones**

#####**Funcion optimiza los metodos MIM y IMC**

In [ ]:

def MIM_IMC():
  #Seleccion del subtipo molecular
  problema_de_clasificacion = {"0":"luminal_A", 
                        "1":"luminal_B",
                        "2":"her2",
                        "3":"triple_negativo",
}
  print(problema_de_clasificacion)
  problemas_elegidos = problema_de_clasificacion[input("Indique el valor del subtipo molecular: ")] 
  


  #Metodo_de_seleccion_de_características
  Metodo_de_seleccion_de_características = { "1":"_IMC_",
                                      "2":"_MIM_",
                                  }
  print(Metodo_de_seleccion_de_características)
  fsel = Metodo_de_seleccion_de_características[input("Indique el valor del método de selección de características:")]

  #ramdom_state
  Random_State_ = {"0":0,"42":42, "50":50, "60":60, "90":90} #Será necesario tantas veces? . Este artículo habla de cómo 5 es suficiente. https://www.sciencedirect.com/science/article/pii/S0957417421013622?via%3Dihub#tbl1
  print(list(Random_State_.keys()))
  Random_State = Random_State_[input("Indique el valor del random state: ")] 

  return problemas_elegidos,fsel,Random_State

#####**Funcion optimiza los metodos IG**

In [ ]:
def IG():
  #Seleccion del subtipo molecular
  problema_de_clasificacion = {"0":"luminal_A", 
                        "1":"luminal_B",
                        "2":"her2",
                        "3":"triple_negativo",
}
  print(problema_de_clasificacion)
  problemas_elegidos = problema_de_clasificacion[input("Indique el valor del subtipo molecular: ")] 
  


  #Metodo_de_seleccion_de_características
  fsel = "_IG_"
  
  #ramdom_state
  Random_State = [0,42,50,60,90]
  
  return problemas_elegidos,fsel,Random_State

#####**Funcion optimiza los metodos IM,RELF y MRMR**

In [ ]:
def IM_RELF_MRMR():
  #Seleccion del subtipo molecular
  problema_de_clasificacion = {"0":"luminal_A", 
                        "1":"luminal_B",
                        "2":"her2",
                        "3":"triple_negativo",
}
  print(problema_de_clasificacion)
  problemas_elegidos = problema_de_clasificacion[input("Indique el valor del subtipo molecular: ")] 
  
  #Metodo_de_seleccion_de_características
  Metodo_de_seleccion_de_características = { "1":"_IM_",
                                      "2":"RELF",
                                      "3": "_MRMR_",

                                  }
  print(Metodo_de_seleccion_de_características)
  fsel = Metodo_de_seleccion_de_características[input("Indique el valor del método de selección de características:")]


  #ramdom_state
  Random_State = [0,42,50,60,90]
  
  return problemas_elegidos,fsel,Random_State

#####**Funcion optimiza los metodos MF, PF,PT y PW**

In [ ]:
def MF_PF_PT_PW():

  #Seleccion del subtipo molecular
  problemas_elegidos = {"0":"luminal_A", 
                        "1":"luminal_B",
                        "2":"her2",
                        "3":"triple_negativo",
}


  #Feature selection methods
  Feature_selection_methods = {"1":"_f_score_","2":"_Fisher_", "3":"_t_score_","4":"_WLCX_","5":"_LASSO_"}
  print(Feature_selection_methods)
  fsel = Feature_selection_methods[input("Indicate the value of the feature selection method: ")]

  #Metodo_de_seleccion_de_características
  Metodo_de_seleccion_de_características = { "1":"_MF_",
                                      "2":"_PF_",
                                      "3": "_PT_",
                                      "4": "_PW_",
                                  }
  print(Metodo_de_seleccion_de_características)
  fsel = Metodo_de_seleccion_de_características[input("Indique el valor del método de selección de características:")]

  #ramdom_state
  Random_State = [0,42,50,60,90]

  
  return problemas_elegidos,fsel,Random_State

####**Metodos de seleccion de caracteristicas**

In [ ]:
def metodos_seleccion_caracteristicas(fsel, n_caracteristicas):
 #Este grupo de métodos de selección de funciones se basa en la biblioteca sklearn
  if fsel == "_MF_":
    fselector = SelectKBest( k = n_feature)
  elif fsel =="_IM_":
    fselector = SelectKBest(mutual_info_classif, k = n_feature)
    
  ##Este grupo de métodos de selección de funciones se basa en la biblioteca skfeature 
  elif fsel == "_PF_":
    def fisher_score_fct (X, y):
      scores = fisher_score.fisher_score (X,y)
      return scores
    fselector = SelectKBest(fisher_score_fct, k = n_feature)
  elif fsel == "_ReliefF_":
    def relieff_score_fct (X, y):
      scores = reliefF.reliefF (X,y)
      return scores
    fselector = SelectKBest(relieff_score_fct, k = n_feature)
  elif fsel == "_IG_":
    def gini_index_fct (X, y):
      scores = gini_index.gini_index (X,y)
      return scores
    fselector = SelectKBest(gini_index_fct, k = n_feature)
  elif fsel == "_PT_":
    def t_score_fct (X, y):
      scores = t_score.t_score (X,y)
      return scores
    fselector = SelectKBest(t_score_fct, k = n_feature)
  elif fsel == "_IMC_":
    def jmi_fct (X, y):
      scores = JMI.jmi (X,y)
      return scores
    fselector = SelectKBest(jmi_fct, k = n_feature)
  elif fsel == "_MIM_":
    def mim_fct (X, y):
      scores = MIM.mim (X,y)
      return scores
    fselector = SelectKBest(mim_fct, k = n_feature)
  elif fsel == "_MRMR_":
    def mrmr_fct (X, y):
      scores = MRMR.mrmr (X,y)
      return scores
    fselector = SelectKBest(mrmr_fct, k = n_feature)
  #Feature Selection method created in the experiment using with reference the code of Aydin.
  elif fsel ==  "_PW_":
    def wilcoxon_score_fct (X, y):
      scores_ = wilcoxon_score (X,y)
      scores = feature_ranking(scores_)
      return scores
    fselector = SelectKBest(wilcoxon_score_fct, k = n_feature)
  return fselector

####**Algoritmo de seleccion de caracteristicas en la base de datos del cancer de mama**

#####**Base de datos cancer de mama**

In [ ]:
#dataframe de las caracteristicas de las imagenes
X = pd.read_excel("/content/drive/MyDrive/TESIS/Experimento_Final/Base_de_datos/Imaging_Features.xlsx")

#toma del dataframe de duke "Clinical_and_Other_Features" y filtra solo las columnas de los receptores hormonales y la clasificacion de subtipo molecular.
df_clinic = pd.read_excel("/content/drive/MyDrive/TESIS/Experimento_Final/Base_de_datos/Clinical_and_Other_Features.xlsx")
y = (df_clinic[['Patient Information',"Tumor Characteristics",'Unnamed: 24','Unnamed: 25', 'Unnamed: 26']]
                     .rename(columns=df_clinic.iloc[0])
                     .drop(1)
                     .iloc[1:]
                     .reset_index(drop=True))
df_base = X.merge(y, on="Patient ID")
df_base

#####**Algoritmo para cancer de mama**

In [ ]:
def Algoritmo_seleccion_caracterisitcas(df_base,problemas_elegidos,fsel,Random_State,n_caracteristicas):
'''PRIMERA PARTE:
  El problema elegido tiene dos formas de análisis:
  - El primero está relacionado con el subtipo molecular del cáncer de mama, donde la columna en el data frame se denomina 'Mol Subtype', en la que encontramos 4 clases, correspondientes a los 4 subtipos moleculares. luminal A, B, HER2 y triple negativo. Para este experimento, trabajaremos con problemas binarios, comparando una clase con las otras tres.
  - El segundo está relacionado con el receptor hormonal, donde las columnas se definen como ER, PR y HER2. Para este caso solo hay dos clases, es decir, es positivo (0) o negativo (1).
  '''

  if chosen_problems == "luminal_A" or "luminal_B" or "her2" or "triple_negative":
    df = df_base.drop(["Patient ID","ER","PR","HER2"], axis=1) #del marco de datos original, las columnas indicadas se eliminan, dejando la columna "subtipo molecular" como objetivo
  
    #Transforma otros subtipos definidos como clase 1, donde la clase 0 corresponde al tipo seleccionado.
    if chosen_problems =="luminal_A":
      df.loc[df['Mol Subtype'] == 2, 'Mol Subtype'] = 1
      df.loc[df['Mol Subtype'] == 3, 'Mol Subtype'] = 1
    elif chosen_problems =="luminal_B":
      df.loc[df['Mol Subtype'] == 0, 'Mol Subtype'] = 4
      df.loc[df['Mol Subtype'] == 1, 'Mol Subtype'] = 0
      df.loc[df['Mol Subtype'] == 2, 'Mol Subtype'] = 4
      df.loc[df['Mol Subtype'] == 3, 'Mol Subtype'] = 4
      df.loc[df['Mol Subtype'] == 4, 'Mol Subtype'] = 1
    elif chosen_problems =="her2":
      df.loc[df['Mol Subtype'] == 0, 'Mol Subtype'] = 4
      df.loc[df['Mol Subtype'] == 1, 'Mol Subtype'] = 4
      df.loc[df['Mol Subtype'] == 2, 'Mol Subtype'] = 0
      df.loc[df['Mol Subtype'] == 3, 'Mol Subtype'] = 4
      df.loc[df['Mol Subtype'] == 4, 'Mol Subtype'] = 1
    else:
      df.loc[df['Mol Subtype'] == 0, 'Mol Subtype'] = 4
      df.loc[df['Mol Subtype'] == 1, 'Mol Subtype'] = 4
      df.loc[df['Mol Subtype'] == 2, 'Mol Subtype'] = 4
      df.loc[df['Mol Subtype'] == 3, 'Mol Subtype'] = 0
      df.loc[df['Mol Subtype'] == 4, 'Mol Subtype'] = 1
    
    '''División Características y target, en las variables X e y respectivamente'''
    # # Para este marco de datos, el objetivo son las columnas  "Mol Subtype"
    y = df["Mol Subtype"]
    X = df.drop(["Mol Subtype"], axis = 1)
  
'''SEGUNDA PARTE: PREPROCESAMIENTO DE DATOS'''

'''Imputadora para completar valores faltantes'''
  #se usó SimpleImputer de las bibliotecas Sklearn.
  simp = SimpleImputer(strategy="mean") # "media", luego reemplace los valores faltantes usando la media a lo largo de cada columna. Solo se puede utilizar con datos numéricos.
  X = pd.DataFrame(simp.fit_transform(X),columns = X.columns)

 '''escalamiento de datos'''
  #se usó StandardScaler de las bibliotecas Sklearn.
  sscal = StandardScaler() #Estandarice las características eliminando la media y escalando a la varianza de la unidad.
  X = pd.DataFrame(sscal.fit_transform(X),columns = X.columns) 
'''Dividir conjunto de datos en entrenamiento y prueba'''
  #se uso train_test_split of the libaries Sklearn.
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.15,random_state=Random_State)

  '''Manejo de clases desbalanceadas'''
  #se utilizó sobremuestreo de las librerías imblearn.
  if over_sampling == "RandomOverSampler":
    ros = RandomOverSampler(random_state=0)
    X_train, y_train = ros.fit_resample(X_train, y_train) 

  
'''TERCERA PARTE: MÉTODO DE SELECCIÓN DE CARACTERÍSTICAS'''

  '''Se aplica el método de selección de caractertisticas.'''
  fselector = metodos_seleccion_caracteristicas(fsel, n_caracteristicas) # funcion metodos de seleccion de caracteristicas
  start = time.time()  # inicia el temporalizador de funcion metodos de seleccion de caracteristicas
  with np.errstate(divide='ignore',invalid='ignore'):
    fselector.fit(X_train, y_train) #Se ejecuta la funcion de seleccion de caracteristicas
  end = time.time() # finaliza el temporalizador de funcion metodos de seleccion de caracteristicas
  times_fselector = str(end - start) #times_fselector, corresponde al tiempo de ejecución del método de selección de características
  
  '''creación de dataframes filtrados'''
  feature_idx = fselector.get_support() #devuelve el índice de las características seleccionadas del dataframe de datos original
  feature_names = X.columns[feature_idx] #devuelve el nombre de las características seleccionadas del dataframe de datos original
  X_train_fs = fselector.transform(X_train) #devuelve una matriz de solo las características seleccionadas del dataframe de datos original, que se usa para el entrenamiento
  X_test_fs = fselector.transform(X_test) #devuelve una matriz de solo las características seleccionadas del dataframe de datos original, usando para probar

  '''''Se crea un nuevo dataframe de datos con las características seleccionadas que se usarán tanto para entrenamiento como para prueba. '''

  feature_names_series = pd.Series(feature_names,copy=False,name =times_fselector) #series con los nombres de las características seleccionadas
  X_train_fs = pd.DataFrame(X_train_fs,columns=feature_names)
  y_train = pd.Series(data = y_train)
  X_test_fs = pd.DataFrame(X_test_fs,columns=feature_names)
  y_test = pd.Series(data = y_test)


  '''Guardar archivos en CSV en Google Drive'''

  #guardar archivos en CSV
  feature_names_series.to_csv("/content/drive/MyDrive/tesis/"+ problemas_elegidos +"/seleccion_caracteristicas/"+fsel+"/_"+str(Random_State)+"/feature_names/"+chosen_problems +"_rs_"+str(Random_State)+"_Nombres_caracteristicas"+fsel+str(n_caracteristicas)+".csv")
  X_train_fs.to_csv("/content/drive/MyDrive/tesis/"+ problemas_elegidos +"/seleccion_caracteristicas/"+fsel+"/_"+str(Random_State)+"/X_train/"+chosen_problems +"_rs_"+str(Random_State)+"_X_train"+fsel+str(n_caracteristicas)+".csv")
  y_train.to_csv("/content/drive/MyDrive/tesis/"+ problemas_elegidos +"/seleccion_caracteristicas/"+fsel+"/_"+str(Random_State)+"/y_train/"+chosen_problems +"_rs_"+str(Random_State)+"_y_train"+fsel+str(n_caracteristicas)+".csv")
  X_test_fs.to_csv("/content/drive/MyDrive/tesis/"+ problemas_elegidos +"/seleccion_caracteristicas/"+fsel+"/_"+str(Random_State)+"/X_test/"+chosen_problems +"_rs_"+str(Random_State)+"_X_test"+fsel+str(n_caracteristicas)+".csv")
  y_test.to_csv("/content/drive/MyDrive/tesis/"+ problemas_elegidos +"/seleccion_caracteristicas/"+fsel+"/_"+str(Random_State)+"/y_test/"+chosen_problems +"_rs_"+str(Random_State)+"_y_test"+fsel+str(n_caracteristicas)+".csv")

#**Aplicacion del experimento**

##**MIM, IMC**

In [ ]:
#selecciona los parametros de la funcion Algoritmo_seleccion_caracterisitcas
problemas_elegidos,fsel,Random_State = MIM_IMC()

In [ ]:
#ejecucion del experimento
for i in range(len(n_feature)):
  Algoritmo_seleccion_caracterisitcas(df_base,problemas_elegidos,fsel,Random_State,n_caracteristicas[i])

##**IG**

In [ ]:
#selecciona los parametros de la funcion Algoritmo_seleccion_caracterisitcas
problemas_elegidos,fsel,Random_State = IG()

In [ ]:
#ejecucion del experimento
for x in range(len(Random_State)):
  for i in range(len(n_feature)):
    Algoritmo_seleccion_caracterisitcas(df_base,problemas_elegidos,fsel,Random_State[x],n_caracteristicas[i]):

##**IM_RELF_MRMR**

In [ ]:
#selecciona los parametros de la funcion Algoritmo_seleccion_caracterisitcas
problemas_elegidos,fsel,Random_State = IM_RELF_MRMR()

In [ ]:
#ejecucion del experimento
for x in range(len(Random_State)):
  for i in range(len(n_feature)):
    Algoritmo_seleccion_caracterisitcas(df_base,problemas_elegidos,fsel,Random_State[x],n_caracteristicas[i]):

##**MF, PF, PT y PW**

In [ ]:
#selecciona los parametros de la funcion Algoritmo_seleccion_caracterisitcas
problemas_elegidos,fsel,Random_State = MF_PF_PT_PW()

In [ ]:
#ejecucion del experimento  
for y in range(len(chosen_problems)):
  for x in range(len(Random_State)):
    for i in range(len(n_feature)):
      Algoritmo_seleccion_caracterisitcas(df_base,problemas_elegidos[y],fsel,Random_State[x],n_caracteristicas[i]):